# Goal of this notebook

Show what tf-idf (term frequency, inverse document frequency) does to terms from documents - and how it is limited.




## What is it?
### For context

Say you are creating the following table: 
- documents are rows
- columns are terms in them
- cells count how often each term appears in each document.


You can imagine something this might be useful to do things like 
- compare documents, by use of the same words,
- perhaps find similar terms - if you know which documents are similar, you can guess which terms might be related.


You can also imagine that ''just'' counts has _issues_ 
- Longer documents would have higher numbers not because terms are more important, but just because there are more words in the document.
- Common words would have high counts just because they are common, not because they are meaningful.
  - In fact, things like [Zipfian word distribution](https://en.wikipedia.org/wiki/Zipf%27s_law) points out that in natural text,
    common terms are ''disproportionally'' common - a handful like _the_, _be_, _to_, _of_, _and_, _in_, _a_, will cover perhaps 20% of all text,
    while being semantically empty function words.

### Basic motivation of tf-idf

tf-idf was introduced in the 1970s, as a heuristic of 'term specificity', based on two rough intuitions:
- a term more common in a document is probably important to that document

but _at the same time_
- a term common to **all** documents is probably less important in general.

- term frequency (tf) is often calculated as `(count of term within document / total terms in document)` - so counts expressed as a ratio.
- document frequency (df) is often calculated as `log( count of documents / count of documents that contain the term )`. 
- The word 'inverse' indicates that we *divide* by this: _IDF_ just means _1 / DF_

Those ways of calculating are the basics. Specific applications may will often try to be smarter, but still call it tf-idf (we mention this because if you use someone else's code, it may well use such an alternative, and not show the same values as a basic tutorial would).

## What does it give you, and what can you do with it?

TF-IDF isn't a method, it isn't a goal. 
It is a tool.

Actually, by itself it's barely even a tool, in that the table/matrix itself is,
for each term, just *data* - a weight for each document a term is in.


The use of that data lies in the application,
and different things may use that data very differently. 

If you are inspecting a collection of documents for what might be more unique and topical terms, 
you might wish to exclude terms appearing in most documents, just to ignore a lot of muck.

If you're creating a field-specific stopword list, or perhaps estimate how _not_ to the point a document is,
the muck is exactly what you are interested in.


Say, if you're using those tf-idf values, e.g. to score things, scores, then you probably want some mathematical logs in there.

### What could you do?

* score results in a search system - idf means putting little weight in things not interested in the document set, tf means focusing more on specifics

* make stopword lists / automatically downweigh repetitive words
  things you would consider stopwords will automatically grow the highest IDF
  and it's a gliding scale with other words that happen to be common

* try to score a document for how repetitive or overcomplete something is

### Limitations

Note that the larger and more heterogenous the document set, the more diluted some effects are,
and some uses stand and fall with a little control.

Say you're comparing all possible media products, then terms like 'film', 'book', 'album' are very distinguishing,
and you may want those to be reported so you can partition them later.

If you're looking at all your products this will be stronger.

If you're looking just within movie reviews, or just within all book reviews, or just album reviews, 
then these are the terms you want to fall away.

IDF, when calculated for such a specific collection, will basically do that for you,
but when you want to do things based on the result, you should be aware of this context sensitivity,
and you may want to do a little more curation.

<!-- -->

You should not assume its output is linear or usable as probabilities - it's ordinal, but distribution is skewed,
so this isn't even very good input to a lot of methods.

It's better than nothing, and e.g. in result scoring the values themselves don't have to make sense.

Applying log will help, but not 

<!-- -->

You should not assume the output lets you do direct comparisons anyway -- 
the numbers you end up with in the table is based on ratios within the document,
which means that longer document have lower values on each term by definition,
which may not match your intuition of how much about certain terms it is.

### Other notes

Whether you see the results of TF-IDF calculation as document-by-term matrix that happens to be sparsely filled, or another way,
depends a little on implementation and intent.

Tutorials out there will differ (it seems each field does its own thing).



# ...Okay, how about some examples?

## TFIDF using sklearn

In [5]:
import numpy
import sklearn, sklearn.feature_extraction

In [6]:
class TfIdf:
    ''' This is mostly leveraging sklearn's existing tfidf code, 
        wrapped in a class to make it slightly easier to feed in documents and inspect the result
    '''
    def __init__(self):
        self.docs = []
        self.docnames = []
        self.doci = 1

    def add_document(self, doc_as_term_sequence, docname=None):
        ' takes a document as a string '
        if docname==None:
            docname = 'doc%d'%self.doci
            self.doci += 1
        self.docnames.append( docname )
        self.docs.append( doc_as_term_sequence )

    def calc(self, **kwargs):
        """ mostly just calls sklearn's TfidfVectorizer.
            
            - min_df:int: ignore terms that appear in fewer than this amount of documents   (default is 2, rather than 1)
            - max_df:float: ignore terms that appear in fewer than this amount of documents   (default is 2, rather than 1)
        """
        min_df = kwargs.pop('min_df',2)
        max_df = kwargs.pop('max_df',1.0)
        ngram_range = kwargs.pop('ngram_range',(1,1))
        max_features = kwargs.pop('max_features',None)
        stop_words = kwargs.pop('stop_words',None)
        sublinear_tf = kwargs.pop('sublinear_tf',False)
        if len(kwargs) > 0:
            print("Unrecognized kwargs: %s"%', '.join(kwargs.keys()))

        # see also https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn-feature-extraction-text-tfidfvectorizer
        self.vectorizer      = sklearn.feature_extraction.text.TfidfVectorizer( 
            min_df=min_df,   # terms that appear in only one document are assumed to be misspelled or otherwise unimportant
            max_df=max_df,
            max_features=max_features,
            ngram_range=ngram_range,
            stop_words=stop_words,
            sublinear_tf=sublinear_tf,
            use_idf=True, # (true by default)
        )
        # the fit part learns the vocabulary and idf,  the tansform part turns documents into document-term matrices
        self.doc_term_matrix = self.vectorizer.fit_transform( raw_documents=self.docs )
        #print(self.doc_term_matrix)

    def vocab_idf(self):
        ' returns (term, internalfeatureindex, IDF) for the entire vocabulary'
        ret = []
        for term, feati in self.vectorizer.vocabulary_.items():
            ret.append( (term, feati, self.vectorizer.idf_[feati]) )
        ret.sort(key=lambda x:x[2])
        return ret


class ShowTfIdf:
    ' pretty-print such a TfIdf objects   in the context of ipython style notebooks '
    def __init__(self, ob):
        self.ob = ob
    
    def _repr_html_(self):
        ret = []
        if not hasattr(self.ob, 'vectorizer'):
            ret.append('<p>not calc()ulated yet</p>')
        else:
            #M = self.ob.doc_term_matrix.todense()
            M = self.ob.doc_term_matrix
            nM = M/numpy.amax(M)
            ret.append('<table>')
            try:
                names = self.ob.vectorizer.get_feature_names()  # older sklearn
            except AttributeError:
                names = self.ob.vectorizer.get_feature_names_out() 
            
            ret.append('<tr><th></th>')
            for name in names:
                ret.append('<th>%s</th>'%name) # TODO: escape
            ret.append('</tr>')

            num_docs, num_terms = nM.shape
            for doci in range( num_docs ):
                ret.append('<tr>')
                ret.append('<th>%s</th>'%self.ob.docnames[doci])
                for termi in range(num_terms):
                    p = nM[doci,termi]
                    ret.append('<td style="opacity:%.1f">%.3f</td>'%(p,p))
                ret.append('</tr>')
            
            ret.append('</table>')
        return ''.join(ret)

### Tiny example
Let's have a tiny small example we can grasp, and that wouldn't be spammy if we printed it

In [8]:
docs = [
    'I am the duck what goes quack',
    'I am a computer program',
    'I am a member of The The',
    'I am the fox',
    'I am the fox in a longer sentence',
]

tfidf = TfIdf()
for doc in docs:
    tfidf.add_document( doc )

tfidf.calc(
    min_df=1,    # defaults to 2, only does anything when it is >2;   1 means no filtering and is intentional for this tiny an example   
    max_df=1.0,
    ) 
ShowTfIdf( tfidf )

,am,computer,duck,fox,goes,in,longer,member,of,program,quack,sentence,the,what
doc1,0.303,0.000,0.636,0.000,0.636,0.000,0.000,0.000,0.000,0.000,0.636,0.000,0.358,0.636
doc2,0.433,0.908,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.908,0.000,0.000,0.000,0.000
doc3,0.345,0.000,0.000,0.000,0.000,0.000,0.000,0.725,0.725,0.000,0.000,0.000,0.817,0.000
doc4,0.591,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.698,0.000
doc5,0.315,0.000,0.000,0.534,0.000,0.662,0.662,0.000,0.000,0.000,0.000,0.662,0.373,0.000


Things of note: 
- 'am' is downscored due to being in every document. ('I' would be too if it weren't removed due to removing _everything_ of length 1)
- 'the' is an example of a term being more common in a document scores it up (for what we suspect is probably no good reason), and also demonstrates that the amount of effect idf should have is perhaps debatable 

- `min_df` removes things that appear in fewer than this many documents (probably misspelled words) - less valuable in this particular example because we have so few documents
- `max_df` removes things that appear in more than this many documents (probably function words, or otherwise so common that they have no distinguishing value)

- a max_df that is <1.0 but still relatively high can be used to remove what you'ld call stopwords - e.g 0.7 would remove 'am' and 'the'

However, we usually don't have terms that are that unique / table that is that sparse. An example on more words:

## Example in unstructured text

In [9]:
import wetsuite.datasets, random
rvs = wetsuite.datasets.load('raadvanstate-adviezen-struc')

count_ignored, count_selected = 0,0
lengths = []
larger_rvs_advice = []
for key, item in rvs.data.items():
    body = '\n'.join( item['body'] )
    lengths.append(len(body))
    # there's a bunch short ones we ignore, in part to get things with _similar_ amount of words. Choice is based on:
    #   df = pandas.DataFrame( {'lens':lengths} )
    #   df[ df["lens"] >900 ][ df["lens"] <30000 ].hist(  )
    if len(body) > 1000 and len(body)<15000: 
        larger_rvs_advice.append( (item['meta'].get('Kenmerk',None), item['url'], body) )
        count_selected += 1
    else:
        count_ignored += 1


selection = random.sample( larger_rvs_advice, 50 )  
# a list of 50   (case_id, url, case_text)  pairs

count_selected, count_ignored

(4355, 7517)

In [10]:
rvs_selection_tfidf = TfIdf()
for kenmerk, url, body in selection:
    #rvs_tfidf.add_document( body, docname=kenmerk )
    rvs_selection_tfidf.add_document( body, docname='<a href="%s">%s</a>'%(url, kenmerk) )   # that HTML-like thing is cheating, but happens to work

In [11]:
# words that seem characteristic of the documents - terms that are neither rare, or everywhere - i.e. those that show up in a middling amount of documents
rvs_selection_tfidf.calc( # some notes on argments below
    min_df=20, max_df=40,
    stop_words=['aan','aanhangig','aanleiding','als','bij', 'daarmee', 'daarbij', 'dan', 'dat', 'der', 'deze', 'die', 'doen', 'door', 'dus', 'er', 'geen', 'geeft', 'gelet', 'gemaakt', 
                'heeft', 'hebben', 'ik', 'ii', 'kan', 'luidt', 'maar', 'naar', 'niet', 'of', 'om', 'ook', 'over', 'te', 'ten', 'ter', 'tot', 'uit', 'voor', 'zal', 'zie', 'zijn']
)

ShowTfIdf( rvs_selection_tfidf )

,26,aangepast,aantal,advisering,afdeling,andere,behorende,besluit,besluiten,betreffende,betrekking,bijlage,dient,eerste,enkele,gebruik,gehele,gehouden,gelegenheid,generaal,gewijzigde,houdende,inzake,kamer,kb,maakt,maken,meer,memorie,na,nadat,nemen,nodig,nog,noot,nota,onder,ontwerp,ontwerpbesluit,ontwerpregeling,opgenomen,opmerkingen,overeenkomstig,redactionele,regeling,rekening,staatssecretaris,staten,tekst,toekomen,toepassing,tweede,verband,verwijst,vijfde,voorgestelde,voorstel,vorenstaande,wetsvoorstel,wijziging,wijzigingen,worden,wordt,zaken,zenden,zesde,zich
W12.17.0150/III,0.075,0.153,0.112,0.152,0.218,0.035,0.000,0.673,0.067,0.000,0.038,0.286,0.000,0.030,0.062,0.070,0.078,0.000,0.036,0.000,0.161,0.035,0.000,0.098,0.038,0.000,0.127,0.030,0.000,0.071,0.000,0.000,0.000,0.077,0.130,0.249,0.062,0.068,0.798,0.039,0.578,0.052,0.032,0.000,0.153,0.000,0.000,0.000,0.070,0.036,0.075,0.078,0.030,0.000,0.038,0.039,0.000,0.000,0.000,0.076,0.244,0.247,0.108,0.065,0.000,0.039,0.000
W06.09.0270/III,0.000,0.000,0.329,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.268,0.337,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.227,0.000,0.242,0.000,0.000,0.000,0.000,0.612,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.280,0.000,0.337,0.000,0.919,0.000,0.237,0.000,0.000,0.000,0.345
W05.01.0150/III,0.000,0.152,0.000,0.000,0.000,0.346,0.066,0.056,0.066,0.068,0.152,0.127,0.152,0.178,0.062,0.000,0.078,0.078,0.000,0.000,0.107,0.138,0.078,0.000,0.000,0.149,0.000,0.182,0.000,0.142,0.063,0.068,0.149,0.076,0.323,0.186,0.186,0.068,0.595,0.000,0.000,0.051,0.127,0.218,0.152,0.071,0.156,0.000,0.000,0.071,0.223,0.308,0.178,0.063,0.000,0.000,0.000,0.063,0.000,0.252,0.277,0.546,0.535,0.000,0.078,0.000,0.000
W07.12.0124/II,0.000,0.000,0.040,0.130,0.631,0.037,0.035,0.060,0.000,0.036,0.041,0.102,0.163,0.191,0.033,0.000,0.000,0.042,0.000,0.036,0.000,0.074,0.000,0.035,0.000,0.119,0.000,0.098,0.122,0.038,0.034,0.000,0.000,0.082,0.416,0.000,0.133,0.000,0.035,0.000,0.109,0.028,0.034,0.059,0.041,0.038,0.000,0.037,0.000,0.000,0.119,0.083,0.127,0.068,0.000,0.293,0.170,0.034,0.898,0.081,0.037,0.351,0.344,0.000,0.042,0.042,0.000
W04.18.0069/I,0.059,0.060,0.000,0.239,0.489,0.492,0.000,0.000,0.000,0.053,0.120,0.000,0.060,0.000,0.049,0.328,0.062,0.000,0.057,0.107,0.084,0.000,0.062,0.153,0.060,0.000,0.300,0.048,0.120,0.000,0.050,0.000,0.000,0.000,0.204,0.000,0.098,0.000,0.000,0.062,0.053,0.041,0.000,0.000,0.240,0.000,0.123,0.328,0.055,0.000,0.000,0.162,0.094,0.000,0.000,0.000,0.600,0.050,0.421,0.080,0.382,0.302,0.127,0.204,0.123,0.000,0.123
WO9.05.0505/V,0.000,0.000,0.160,0.000,0.000,0.224,0.071,0.000,0.000,0.073,0.000,0.683,0.000,0.128,0.067,0.149,0.084,0.000,0.157,0.146,0.116,0.000,0.000,0.140,0.082,0.080,0.000,0.065,0.658,0.229,0.068,0.073,0.000,0.000,0.000,0.000,0.201,0.000,0.000,0.000,0.073,0.000,0.000,0.236,0.082,0.000,0.000,0.149,0.224,0.000,0.000,0.166,0.128,0.068,0.082,0.000,0.273,0.068,0.493,0.381,0.000,0.236,0.000,0.000,0.169,0.000,0.084
W06.21.0121/III,0.046,0.047,0.046,0.113,0.540,0.043,0.000,0.139,0.041,0.000,0.000,0.000,0.047,0.259,0.039,0.000,0.000,0.000,0.000,0.000,0.067,0.000,0.000,0.040,0.000,0.092,0.000,0.113,0.000,0.044,0.000,0.000,0.092,0.000,0.443,0.308,0.386,0.042,0.823,0.000,0.042,0.064,0.039,0.034,0.095,0.044,0.097,0.000,0.086,0.044,0.185,0.128,0.111,0.000,0.047,0.194,0.039,0.000,0.047,0.125,0.086,0.102,0.500,0.000,0.000,0.000,0.000
W02.07.0421/II,0.082,0.000,0.000,0.000,0.000,0.076,0.146,0.062,0.073,0.075,0.084,0.140,0.000,0.066,0.000,0.076,0.000,0.086,0.000,0.448,0.118,0.153,0.086,0.071,0.084,0.082,0.070,0.067,0.000,0.000,0.070,0.075,0.000,0.084,0.143,0.205,0.000,0.075,0.512,0.086,0.075,0.057,0.140,0.181,0.000,0.078,0.172,0.000,0.000,0.078,0.164,0.114,0.262,0.070,0.084,0.517,0.000,0.070,0.168,0.334,0.076,0.181,0.473,0.500,0.000,0.000,0.345
W06.05.0032/IV,0.000,0.065,0.000,0.000,0.000,0.059,0.057,0.000,0.000,0.116,0.455,0.162,0.000,

Notes:
- `min_df=20, max_df=40` speaks more to the imagination more, because we know how many documents we added; 
  - `min_df=0.4, max_df=0.8` would be equivalent in this case, and would keep doing the same with different amounts of documents
- you probably want to play with these numbers, stopwords, and more, to get a feel for how much the output varies. 
  - because while this looks decent, but much of this notebook involves some parameter wrangling

In [12]:
# things that show up more than once (probably not _just_ a typo) but still only in a few documents
rvs_selection_tfidf.calc(min_df=2, max_df=5) 
# there over a thousand terms, though, and this view will be sluggish because of it. 
# You could consider something like max_features=200 to get that somewhat under control, though it defeats part of the point we're about to make
ShowTfIdf( rvs_selection_tfidf )

Notes:
- there are many distinct terms, but very few uses of each. That sparsity makes comparison pretty hard
- you could stick to just the most common terms -- but what that removes _completely_ (that is, the bulk) can have no further effect on calculations
- this will contain _primarily_ fairly-pointless terms  (the long tail of the Zipfian distribution)
- If you _do_ spend a little more time trying and scrolling, you start to notice that while the _very_ bottom of 'things that show up in at most a few documents' is muck
  - yet at some point it transitions to also contain interesting-but-uncommon.  It turns out you can't really tell them apart from just this.

In [13]:
# things that are very common - will generally be what you'ld call stopwords,   
#   ...or, say, anything that is standard in a header because that's on _every_ page.  Or otherwise not distinguish much.
rvs_selection_tfidf.calc(min_df=40, sublinear_tf=True)
ShowTfIdf( rvs_selection_tfidf )

,aan,aanhangig,advies,adviseert,artikel,bij,dat,de,die,dit,een,en,geeft,geen,gemaakt,heeft,het,hierbij,ik,in,is,kabinetsmissive,lid,majesteit,met,minister,moge,nader,no,om,op,over,overweging,president,raad,rapport,reactie,state,te,ter,toelichting,tot,uwe,van,verzoeken,vice,voor,voordracht,wet,wijziging,zijn
W12.17.0150/III,0.466,0.123,0.337,0.000,0.328,0.439,0.585,0.690,0.303,0.379,0.511,0.577,0.280,0.235,0.208,0.208,0.623,0.235,0.334,0.609,0.557,0.128,0.493,0.128,0.484,0.230,0.136,0.118,0.123,0.467,0.467,0.408,0.204,0.123,0.349,0.126,0.258,0.349,0.479,0.123,0.415,0.402,0.128,0.664,0.136,0.123,0.493,0.126,0.139,0.303,0.444
W06.09.0270/III,0.292,0.000,0.796,0.000,0.281,0.000,0.310,0.965,0.000,0.467,0.626,0.265,0.000,0.000,0.000,0.000,0.847,0.000,0.000,0.633,0.671,0.000,0.292,0.000,0.449,0.304,0.000,0.265,0.000,0.000,0.556,0.000,0.000,0.000,0.449,0.281,0.276,0.449,0.265,0.000,0.281,0.281,0.000,1.000,0.000,0.000,0.292,0.000,0.525,0.000,0.000
W05.01.0150/III,0.434,0.133,0.274,0.153,0.520,0.454,0.441,0.704,0.480,0.317,0.533,0.544,0.244,0.390,0.133,0.347,0.589,0.253,0.138,0.554,0.483,0.138,0.466,0.138,0.394,0.000,0.147,0.128,0.225,0.450,0.474,0.357,0.274,0.133,0.434,0.136,0.133,0.334,0.474,0.133,0.399,0.136,0.138,0.685,0.147,0.133,0.549,0.136,0.390,0.406,0.459
W07.12.0124/II,0.498,0.121,0.282,0.234,0.472,0.467,0.588,0.699,0.565,0.398,0.508,0.518,0.221,0.354,0.121,0.315,0.631,0.000,0.000,0.567,0.485,0.125,0.423,0.125,0.430,0.133,0.000,0.243,0.204,0.408,0.498,0.354,0.248,0.121,0.303,0.000,0.000,0.303,0.430,0.204,0.362,0.418,0.125,0.660,0.000,0.121,0.483,0.123,0.400,0.297,0.504
W04.18.0069/I,0.494,0.134,0.131,0.366,0.136,0.419,0.497,0.711,0.558,0.412,0.610,0.507,0.145,0.359,0.281,0.319,0.644,0.150,0.363,0.542,0.496,0.139,0.142,0.139,0.448,0.000,0.148,0.218,0.134,0.473,0.468,0.255,0.222,0.134,0.307,0.136,0.134,0.270,0.514,0.281,0.523,0.356,0.139,0.693,0.148,0.134,0.482,0.136,0.393,0.265,0.437
WO9.05.0505/V,0.368,0.146,0.299,0.283,0.457,0.398,0.483,0.738,0.445,0.305,0.267,0.499,0.330,0.000,0.305,0.246,0.633,0.164,0.151,0.566,0.504,0.151,0.324,0.151,0.412,0.272,0.161,0.140,0.246,0.436,0.334,0.164,0.299,0.146,0.499,0.148,0.146,0.365,0.462,0.347,0.475,0.475,0.151,0.742,0.161,0.146,0.475,0.148,0.504,0.502,0.289
W06.21.0121/III,0.486,0.129,0.265,0.352,0.517,0.428,0.492,0.712,0.497,0.336,0.569,0.540,0.236,0.245,0.129,0.270,0.624,0.145,0.134,0.544,0.486,0.134,0.381,0.134,0.431,0.000,0.142,0.259,0.129,0.412,0.511,0.245,0.213,0.129,0.295,0.131,0.129,0.295,0.474,0.129,0.553,0.313,0.134,0.685,0.142,0.129,0.552,0.131,0.404,0.359,0.421
W02.07.0421/II,0.459,0.222,0.335,0.255,0.454,0.425,0.566,0.705,0.491,0.275,0.604,0.428,0.240,0.148,0.131,0.275,0.591,0.148,0.231,0.541,0.504,0.136,0.445,0.136,0.459,0.345,0.145,0.126,0.222,0.481,0.428,0.000,0.270,0.222,0.467,0.134,0.131,0.371,0.483,0.313,0.442,0.349,0.136,0.675,0.145,0.131,0.428,0.134,0.148,0.429,0.429
W06.05.0032/IV,0.406,0.124,0.359,0.299,0.486,0.375,0.475,0.692,0.448,0.324,0.457,0.458,0.351,0.237,0.210,0.297,0.569,0.140,0.361,0.569,0.478,0.129,0.459,0.129,0.471,0.137,0.137,0.202,0.210,0.439,0.518,0.294,0.291,0.124,0.471,0.127,0.124,0.312,0.477,0.124,0.478,0.405,0.129,0.665,0.137,0.124,0.448,0.127,0.519,0.305,0.380
W03.08.0269/II/K,0.562,0.172,0.169,0.198,0.000,0.403,0.571,0.807,0.202,0.292,0.445,0.625,0.186,0.194,0.292,0.361,0.699,0.194,0.179,0.488,0.490,0.179,0.000,0.179,0.347,0.453,0.190,0.166,0.172,0.198,0.462,0.000,0.286,0.172,0.488,0.176,0.172,0.395,0.547,0.172,0.517,0.419,0.179,0.832,0.190,0.172,0.562,0.176,0.000,0.000,0.423


Notes:
- this is in itself _almost_ a stopword list. 
- a max_df that is <1.0 and smaller can be used to ignore these

In fact, it's not the worst idea to use exactly this to _create_ a stopword list like this (that adapts to the documents you've fed it):

In [40]:
# we've been using the set of 50 until now,  let's switch to the larger set for better word calculations

rvs_all_tfidf = TfIdf() 

for kenmerk, url, body in larger_rvs_advice:
    #rvs_tfidf.add_document( body, docname=kenmerk )
    rvs_all_tfidf.add_document( body, docname='<a href="%s">%s</a>'%(url, kenmerk) )

display( len(larger_rvs_advice) ) # reminder about the amount of documents

rvs_all_tfidf.calc(min_df=0.5)
common = list( word  for word,_,_ in rvs_all_tfidf.vocab_idf() )
display( len(common) )
display( common )


4355

96

['van',
 'de',
 'op',
 'raad',
 'state',
 'het',
 'en',
 'te',
 'met',
 'advies',
 'in',
 'dit',
 'tot',
 'bij',
 'nader',
 'gemaakt',
 'reactie',
 'heeft',
 'artikel',
 'rapport',
 'ter',
 'is',
 'toelichting',
 'overweging',
 'president',
 'vice',
 'aanhangig',
 'ik',
 'aan',
 'no',
 'voordracht',
 'een',
 'voor',
 'majesteit',
 'uwe',
 'dat',
 'kabinetsmissive',
 'lid',
 'minister',
 'verzoeken',
 'wet',
 'moge',
 'geeft',
 'hierbij',
 'geen',
 'adviseert',
 'wordt',
 'niet',
 'zijn',
 'om',
 'die',
 'tweede',
 'wijziging',
 'naar',
 'gewijzigde',
 'opmerkingen',
 'deze',
 'over',
 'worden',
 'door',
 'verband',
 'kan',
 'zal',
 'aanleiding',
 'of',
 'ook',
 'als',
 'eerste',
 'onder',
 'besluit',
 'dan',
 'nemen',
 'maken',
 'afdeling',
 'uit',
 'redactionele',
 'advisering',
 'nota',
 'voorstel',
 'nadat',
 'maar',
 'overeenkomstig',
 'zie',
 'vorenstaande',
 'kamer',
 'der',
 'staten',
 'ten',
 'meer',
 'generaal',
 'gelet',
 'noot',
 'zaken',
 'doen',
 'ontwerpbesluit',
 'betref

## Do n-grams help?

N-grams (contiguous sequence of things (often words) of length n) encode some context -- but will also count even more sparsely than the things they are based on.

Depending on what you are doing, the last can be a bug, or a feature.

For example, it can be a good way of identifying common phrases - be a simple approximation of collocation).

To return to "seen moderately much" can be more interesting around n-grams, 
because this amounts to "which works  appear together" and becomes a crude form of collocation analysis,
though due to context of legal text, this finds more standard phrasing than it finds interesting terms.

In [41]:
rvs_selection_tfidf.calc(min_df=5, max_df=30, ngram_range=(3,3), sublinear_tf=True, max_features=100)  # using actual numbers out of the 25 speaks to the imagination more 
                                      # min_df=0.4, max_df=0.6 would be equivalent in this case, and work for other amounts of documents
ShowTfIdf( rvs_selection_tfidf )


,aan de tweede,aan te passen,aanleiding tot het,advies behorende bijlage,advisering van de,afdeling advisering van,artikel tweede lid,behorende bijlage de,besluit te nemen,besluiten de minister,bij de afdeling,bij de raad,bij het advies,bijlage bij het,de afdeling adviseert,de afdeling advisering,de bij het,de europese unie,de gewijzigde nota,de memorie van,de nota van,de raad adviseert,de staatssecretaris van,de toelichting op,de tweede kamer,de wet op,der staten generaal,dit ontwerp te,doen toekomen en,een redactionele kanttekening,eerste lid van,en verzoeken overeenkomstig,gemaakt het ontwerpbesluit,gewijzigde nota van,het advies behorende,het maken van,het voorgestelde artikel,het voorstel van,houdende wijziging van,ik moge hierbij,in artikel onderdeel,in de memorie,in de toelichting,in overweging geeft,kamer der staten,lid van de,lid van het,maken van inhoudelijke,memorie van toelichting,met betrekking tot,met memorie van,met nota van,moge hierbij het,naar aanleiding van,naar de bij,nota van toelichting,ontwerp te besluiten,ontwerpbesluit en de,ontwerpregeling met toelichting,op de raad,op dit punt,op grond van,overeenkomstig dit ontwerp,rekening zal zijn,state betreffende no,state bijlage bij,state geeft in,te besluiten de,toekomen en verzoeken,toelichting bij kabinetsmissive,toelichting doen toekomen,tot het maken,tot wijziging van,tweede kamer der,tweede lid van,van de afdeling,van de europese,van de gelegenheid,van het besluit,van het koninkrijk,van het ontwerpbesluit,van het wetsvoorstel,van state adviseert,van state betreffende,van state bijlage,van state geeft,van state van,van toelichting aan,van toelichting bij,van toelichting doen,van toelichting het,verband met de,verzoeken overeenkomstig dit,voorstel van wet,wet op de,wijziging van de,wijziging van het,zal zijn gehouden,zenden aan de,zie noot de
W12.17.0150/III,0.000,0.000,0.334,0.000,0.367,0.367,0.000,0.000,0.000,0.177,0.144,0.000,0.147,0.000,0.000,0.367,0.000,0.000,0.165,0.000,0.514,0.000,0.000,0.000,0.249,0.161,0.000,0.157,0.161,0.000,0.000,0.161,0.157,0.165,0.000,0.316,0.000,0.000,0.192,0.157,0.000,0.000,0.000,0.000,0.000,0.141,0.466,0.343,0.000,0.192,0.000,0.157,0.182,0.000,0.000,0.484,0.157,0.293,0.177,0.187,0.000,0.000,0.157,0.000,0.000,0.000,0.000,0.157,0.161,0.000,0.177,0.324,0.161,0.000,0.000,0.000,0.000,0.169,0.487,0.000,0.334,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.169,0.177,0.165,0.000,0.161,0.000,0.173,0.000,0.286,0.000,0.000,0.187
W06.09.0270/III,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.647,0.000,0.000,0.000,0.000,0.000,0.557,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.487,0.000,0.000,0.599,0.000,0.000,0.000,0.000,0.702,0.000,0.000,0.000,0.000,0.000,0.647,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.630,0.599,0.000,0.000,0.000,0.000,0.000
W05.01.0150/III,0.000,0.188,0.000,0.173,0.000,0.000,0.000,0.173,0.173,0.000,0.000,0.173,0.243,0.147,0.000,0.000,0.173,0.000,0.161,0.000,0.000,0.204,0.301,0.357,0.000,0.000,0.000,0.154,0.157,0.394,0.211,0.157,0.154,0.161,0.173,0.000,0.000,0.000,0.318,0.154,0.507,0.000,0.273,0.169,0.000,0.233,0.218,0.000,0.000,0.000,0.000,0.260,0.178,0.336,0.147,0.323,0.154,0.169,0.000,0.000,0.346,0.193,0.154,0.188,0.000,0.173,0.147,0.154,0.157,0.000,0.173,0.000,0.000,0.000,0.000,0.000,0.233,0.000,0.000,0.000,0.503,0.000,0.000,0.000,0.173,0.144,0.242,0.000,0.000,0.173,0.000,0.286,0.157,0.000,0.000,0.000,0.280,0.188,0.000,0.309
W07.12.0124/II,0.157,0.000,0.000,0.169,0.320,0.320,0.235,0.169,0.000,0.000,0.137,0.000,0.237,0.143,0.000,0.320,0.169,0.000,0.000,0.301,0.000,0.000,0.000,0.000,0.140,0.000,0.150,0.000,0.000,0.000,0.347,0.000,0.000,0.000,0.169,0.000,0.493,0.286,0.309,0.000,0.398,0.000,0.157,0.165,0.150,0.320,0.000,0.000,0.345

## Can we use that IDF number?

In [ ]:
rvs_selection_tfidf.calc(min_df=2, max_df=5 ) 

for term, _, idf in rvs_selection_tfidf.vocab_idf():
    print( '%.3f   %s'%( idf, term) )

# Note that 
# - IDF is lower for emptier terms like  am, the
# - this score isn't affected by how often the term appears in a single document or overall, 
#   just by how many documents it appears in.

### Example in more structured data

Let's look at 20newgroups. 

We know the category of each message, so we can e.g. see how much term IDF varies between categories.

In [49]:
#def simple_tokenize(text: str):
#    ' split string into words - in a very dumb way but slightly better than just split() '
#    import re
#    l = re.split('[\\s!@#$%^&*":;/,?\xab\xbb\u2018\u2019\u201a\u201b\u201c\u201d\u201e\u201f\u2039\u203a\u2358\u275b\u275c\u275d\u275e\u275f\u2760\u276e\u276f\u2e42\u301d\u301e\u301f\uff02\U0001f676\U0001f677\U0001f678-]+', text)
#    return list(e.strip("'")   for e in l  if len(e)>0)            


import pandas # we'll use a dataframe to put the result in, just to get it to show us a table
df = pandas.DataFrame()

from sklearn.datasets import fetch_20newsgroups  # is easy enough to download and use
for category in ('alt.atheism', 'talk.religion.misc', 'sci.space','comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'rec.sport.baseball', 'rec.sport.hockey'):
    #print( "== %s =="%category )
    tfidf = TfIdf()
    for doc in fetch_20newsgroups(subset='train', categories=[category]).data:
        headers, body = doc.split('\n\n',1) # take off newsgroup-like header
        tfidf.add_document( body )

    tfidf.calc()
    ct = []
    ii = []
    for term, _, idf in tfidf.vocab_idf()[:100]: 
        ct.append( term )
        #print( '%.3f   %s'%( idf, term) )
        ii.append( round(idf, 2) )

    df[ category ] = ct
    df[ category+'_idf' ] = ii


df


,alt.atheism,alt.atheism_idf,talk.religion.misc,talk.religion.misc_idf,sci.space,sci.space_idf,comp.sys.ibm.pc.hardware,comp.sys.ibm.pc.hardware_idf,comp.sys.mac.hardware,comp.sys.mac.hardware_idf,rec.sport.baseball,rec.sport.baseball_idf,rec.sport.hockey,rec.sport.hockey_idf
0,the,1.02,the,1.04,the,1.05,the,1.09,the,1.07,the,1.07,the,1.07
1,of,1.08,of,1.10,to,1.12,to,1.18,to,1.16,in,1.16,in,1.17
2,in,1.08,in,1.11,in,1.12,and,1.21,is,1.26,to,1.23,to,1.18
3,to,1.09,to,1.11,of,1.14,of,1.28,and,1.27,and,1.28,and,1.23
4,and,1.12,is,1.13,and,1.20,it,1.29,of,1.28,of,1.31,of,1.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,very,2.50,may,2.42,way,2.68,new,2.88,into,3.03,runs,2.75,were,2.61
96,now,2.50,did,2.46,your,2.70,used,2.90,way,3.03,hit,2.76,should,2.62
97,make,2.52,very,2.49,go,2.70,scsi,2.90,good,3.03,because,2.76,next,2.63
98,get,2.52,fact,2.50,does,2.71,drives,2.92,something,3.04,them,2.76,did,2.64


So what does that give us?

Looks like an automatic stopword generator. Which doesn't have to be aware of the language, or context -- it's not considering the text as text at all.

Yet it's not a lot more. Once you get beyond 100, 200 it's a mix of everything.

And it's not about choosing a cutoff point, because "we don't think about the word, just about in how many documents it appears at all"
is just too crude to (in an admittedly not very large dataset) tell apart that, say, 'yes' and 'christian' have different value. Or 'powerbook' and 'hi'.
and (if you were asking "maybe it's not linear but still fairly ordinal?"), this mix also means it's also too crude for "this term is more specific without telling you how much".